In [1]:
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from dotenv import load_dotenv
_ = load_dotenv()

from langchain_core.messages import SystemMessage, HumanMessage
from agents.supervisor_agent import ChatSupervisorAgent

# from langgraph.graph import StateGraph, END
# from typing import TypedDict, Annotated
# from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
# from langchain_openai import AzureChatOpenAI
# from langchain_community.tools import DuckDuckGoSearchRun

Load Azure Models

In [2]:
from agents.llm_model import AzureModelProvider
provider = AzureModelProvider()

llm = provider.get_primary_model()
fast_llm = provider.get_light_model()

In [3]:
llm.invoke([SystemMessage(content="Ready to assist?")]).content

'Hello! How can I assist you today?'

Import Prompts

In [4]:
# Load the intent classifier prompt from the text file
with open(os.path.join(parent_dir, "prompts", "intent_classifier_prompt.txt"), "r") as f:
    intent_classifier_prompt = f.read()

# Load the action execution prompt from the text file
with open(os.path.join(parent_dir, "prompts", "action_execution_prompt.txt"), "r") as f:
    action_execution_prompt = f.read()


## Agent Set-up

In [5]:
agent = ChatSupervisorAgent(model=llm, system_prompt=intent_classifier_prompt)

In [6]:
# message = "what's my schedule for next week?"
message = "How can I call in sick?"
message = "What's my last payslip?"
# message = "What's Trump's first name?"

In [7]:
test_state = {
    "messages": [HumanMessage(content=message)],
    "candidate_id": os.environ.get("KENTRO_TEST_CANDIDATE_ID"),
    "employee_number": os.environ.get("PLANBITION_TEST_EMPLOYEE_NUM"),
    "next_action": "",
    "retrieved_data": "",
    "error_message": "",
    "attempts": 0,
    "max_retries": 3
}

test_state

{'messages': [HumanMessage(content="What's my last payslip?", additional_kwargs={}, response_metadata={})],
 'candidate_id': '100102195',
 'employee_number': 'RE0036443',
 'next_action': '',
 'retrieved_data': '',
 'error_message': '',
 'attempts': 0,
 'max_retries': 3}

In [8]:
# Run
result = agent.graph.invoke(test_state)

Intent classified as: ActionExecutionAgent
---ACTION EXECUTION AGENT: Starting run()---
---Retrieved context: candidate_id=100102195, employee_id=unknown---
---Last message type: HumanMessage---
---Last message content: What's my last payslip?...---
---Prepared 2 messages for LLM---
---System prompt length: 562 characters---
---Calling LLM with tools---
---LLM response received---
---Response type: AIMessage---
agent_response.tool_calls: [{'name': 'get_payslip', 'args': {'candidate_id': '100102195', 'latest': True}, 'id': 'call_ZhHtk04kycdx15vY897UyFAF', 'type': 'tool_call'}]
---Tool calls count: 1---
---Executing Tool: get_payslip---
---Tool arguments: {'candidate_id': '100102195', 'latest': True}---
---Invoking tool executor---
---LLM response received---
---Response type: AIMessage---
agent_response.tool_calls: [{'name': 'get_payslip', 'args': {'candidate_id': '100102195', 'latest': True}, 'id': 'call_ZhHtk04kycdx15vY897UyFAF', 'type': 'tool_call'}]
---Tool calls count: 1---
---Exec

In [9]:
result["retrieved_data"]

"id=1223595 business_unit_id=10023 serial_number='25-46-001' frequency='W' entry_date='2025-11-19T00:12:52' amount=448.97"

In [10]:
agent.__getstate__

<function ChatSupervisorAgent.__getstate__()>